In [1]:
import pandas as pd
import requests
import time
import os

# Configuration for Qwen2.5 14B
qwen_model = "qwen2.5:14b"
ollama_url = "http://................/api/generate" #Replace with a valid URL
headers = {"Content-Type": "application/json"}

# Instructions for Qwen
instructions = (
    "You will see a prompt containing a sentence with a placeholder [ ] and four options labeled A, B, C, and D.\n"
    "Your task is strictly to select the one best-fitting option to fill the placeholder.\n"
    "Your response must ONLY be one single letter: A, B, C, or D.\n"
    "DO NOT provide explanations, reasoning, or any other text. ONLY one letter."
)

# Call Qwen and validate response
def call_qwen(prompt: str) -> str:
    payload = {
        "model": qwen_model,
        "prompt": f"{instructions}\n\n{prompt}",
        "stream": False,
        "options": {"temperature": 0}
    }

    for attempt in range(3):
        try:
            response = requests.post(ollama_url, json=payload, headers=headers)
            response.raise_for_status()
            result = response.json()["response"].strip().upper()
            if result in ["A", "B", "C", "D"]:
                return result
            else:
                print(f"⚠️ Invalid response '{result}', retrying...")
                return "Invalid"
        except Exception as e:
            print(f"❌ Error: {e} (Attempt {attempt + 1})")
            time.sleep(5)
    return "Error"

# Process file for Qwen
def process_file_qwen(filename):
    df = pd.read_csv(filename)
    output_file = filename.replace(".csv", "_qwen2.5_14b_output.csv")

    if os.path.exists(output_file):
        df_out = pd.read_csv(output_file)
        start_index = df_out["Qwen2.5 Choice"].notna().sum()
    else:
        df["Qwen2.5 Choice"] = ""
        df_out = df.copy()
        start_index = 0

    print(f"📝 Starting from row {start_index} for {filename}")

    for i in range(start_index, len(df_out)):
        row = df_out.iloc[i]
        prompt_text = (
            f"Domain: {row['Domain']}\n"
            f"Prompt: {row['Prompt']}\n"
            f"A: {row['A']}\n"
            f"B: {row['B']}\n"
            f"C: {row['C']}\n"
            f"D: {row['D']}\n"
            "Which option best completes the sentence? "
            "Respond ONLY with one letter: A, B, C, or D."
        )
        choice = call_qwen(prompt_text)
        df_out.at[i, "Qwen2.5 Choice"] = choice

        df_out.to_csv(output_file, index=False, encoding="utf-8-sig")
        print(f"✅ [{i+1}/{len(df_out)}] Saved choice: {choice}")

    print(f"🎉 Finished and saved to {output_file}")
    return df_out

# Run on each language file
for file in ["High.csv", "Mid.csv", "Low.csv"]:
    process_file_qwen(file)


📝 Starting from row 0 for High.csv
✅ [1/60] Saved choice: A
✅ [2/60] Saved choice: B
✅ [3/60] Saved choice: A
✅ [4/60] Saved choice: C
✅ [5/60] Saved choice: B
✅ [6/60] Saved choice: D
✅ [7/60] Saved choice: C
✅ [8/60] Saved choice: D
✅ [9/60] Saved choice: B
✅ [10/60] Saved choice: B
✅ [11/60] Saved choice: B
✅ [12/60] Saved choice: D
✅ [13/60] Saved choice: A
✅ [14/60] Saved choice: B
✅ [15/60] Saved choice: B
✅ [16/60] Saved choice: C
✅ [17/60] Saved choice: C
✅ [18/60] Saved choice: C
✅ [19/60] Saved choice: C
✅ [20/60] Saved choice: D
✅ [21/60] Saved choice: D
✅ [22/60] Saved choice: C
✅ [23/60] Saved choice: B
✅ [24/60] Saved choice: A
✅ [25/60] Saved choice: B
✅ [26/60] Saved choice: A
✅ [27/60] Saved choice: B
✅ [28/60] Saved choice: A
✅ [29/60] Saved choice: C
✅ [30/60] Saved choice: B
✅ [31/60] Saved choice: B
✅ [32/60] Saved choice: D
✅ [33/60] Saved choice: D
✅ [34/60] Saved choice: D
✅ [35/60] Saved choice: A
✅ [36/60] Saved choice: C
✅ [37/60] Saved choice: B
✅ [38/60] Sa